# Get Max Weather Values

Get max weather values for each weather variable over the entire time period for scaling

### Load Packages

In [49]:
import io
import os
import boto3
import pandas as pd
import pickle
import numpy as np

### Variables

In [35]:
###############
## VARIABLES ##
###############

# s3 config
s3_client = boto3.client('s3')
bucket_name = 'hotzone'

# name of directory with weather data
weather_directory = 'weather_data'

# name of key to save max values to
key_name = 'BayAreaWeather/max_values/max_values.pickle'

### Functions to Pull and Save Data to/from S3

In [36]:
def pull_weather_data_from_s3(year, weather_directory):
    '''
    Pull files from S3 for the provided year and save to local directory
    '''
    
    file_name = "weather_data.pickle"
        
    s3 = boto3.resource('s3')
    
    key = "BayAreaWeather/" + str(year) + "/" + file_name
    path = '/home/ubuntu/wildfireplus/weather_data/' + file_name
        
    s3.Bucket('hotzone').download_file(key, path)

In [45]:
def save_array_to_s3(s3_client, array, bucket_name, key_name):
    '''
    Uploads pre-processed data to S3.

    Args:
        - s3_client: boto3 s3 client
        - array: numpy array to save to s3
        - bucket_name: name of bucket on s3 to save array to
        - key_name: directory/file_name to save data to
    Returns:
        - Nothing
    
    https://stackoverflow.com/questions/48049557/how-to-write-npy-file-to-s3-directly
    '''
    
    array_data = io.BytesIO()
    pickle.dump(array, array_data)
    array_data.seek(0)
    
    s3_client.upload_fileobj(array_data, bucket_name, key_name)

### Function to Calculate the Max Weather Value for Each Year

In [38]:
def get_max(weather_directory):
    
    '''
    Iterate over all years and get max value for each attribute
    '''
    
    path = os.path.abspath(weather_directory)

    weather_file = ''

    for f in os.listdir(path):
        if f.endswith('.pickle'):
            weather_file = path + '/' + f

    weather = pd.read_pickle(weather_file)

    for day, attrs in weather.items():
        for attr, mat in attrs.items():
            mat = np.nan_to_num(mat)
            
            if attr in ['High T', 'Low T']:
                mat += 273.15
            
            if mat.max() > max_values[attr]:
                max_values[attr] = mat.max()

### Function to Pull Weather Maxes from S3

In [63]:
def pull_weather_maxes_from_s3():
    '''
    Pull files from S3 for the provided year and save to local directory
    '''
    
        
    s3 = boto3.resource('s3')
    
    key = "BayAreaWeather/max_values/max_values.pickle"
    directory = '/home/ubuntu/wildfireplus/weather_max/'
    path = directory + 'max_values.pickle'
    
    s3.Bucket('hotzone').download_file(key, path)
    
    total_path = os.path.abspath(directory)
    
    for f in os.listdir(total_path):
        if f.endswith('.pickle'):
            max_values = total_path + '/' + f

    max_values = pd.read_pickle(max_values)
    
    return max_values

### Run It

In [43]:
years = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

max_values = {
    'rainint': 0, 
    'raintot': 0, 
    'High T': 0, 
    'Low T': 0, 
    'Humidity': 0, 
    'Wind Speed': 0, 
    'Wind Direction': 0, 
    'Cloud Cover': 0
}

for y in years:
    pull_weather_data_from_s3(y, weather_directory)

    get_max(weather_directory)

In [44]:
# take a look
print(max_values)

{'rainint': 96.67113881170407, 'raintot': 74.02488986783878, 'High T': 319.74086664358845, 'Low T': 304.52986770355284, 'Humidity': 1.0, 'Wind Speed': 62.57522675889799, 'Wind Direction': 359.0000000000001, 'Cloud Cover': 1.0}


In [50]:
# save to s3
save_array_to_s3(s3_client, max_values, bucket_name, key_name)

In [64]:
# pull from s3
max_values = pull_weather_maxes_from_s3()

In [65]:
# take a look
max_values

{'rainint': 96.67113881170407,
 'raintot': 74.02488986783878,
 'High T': 319.74086664358845,
 'Low T': 304.52986770355284,
 'Humidity': 1.0,
 'Wind Speed': 62.57522675889799,
 'Wind Direction': 359.0000000000001,
 'Cloud Cover': 1.0}